## Código desenvolvido em Java para criação da aplicação que utilize a base de dados compras.txt e lista para cada cidade o tipo de produto mais vendido. Para realizar esta tarefa foi desenvolivido dois pares de MapReduce, o primeiro armazena cidade e produto e uma mesma string (chave) dividida por & e coloca no array (valor) um 1 para cada vez que se repete e conta a quantidades de 1 no array de cada chave cidade&produto. A segunda iteração MapReduce lista o produto mais vendido para cada cidade. Projeto inicial chamado contamaxcidadeproduto contendo a estrutura-base das classes da primeira iteração e contamaxcidadeprodutosecond para a segunda iteração MapReduce.

### 1-Desenvolvimento das 3 classes Mapper, Reducer e Driver para implementação da 1º iteração MapReduce

In [ ]:
// Desenvolvimento da classe Driver ContaMaxCidadeProdutoDriver.java

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

public class ContaMaxCidadeProdutoDriver {

	public static void main(String[] args) throws Exception {

		Configuration conf = new Configuration();
		Job job = Job.getInstance(conf, "contamaxcidadeprodutodriver");
		job.setJarByClass(ContaMaxCidadeProdutoDriver.class);
		job.setMapperClass(ContaMaxCidadeProdutoMapper.class);
		job.setMapOutputKeyClass(Text.class);
		job.setMapOutputValueClass(IntWritable.class);
		job.setReducerClass(ContaMaxCidadeProdutoReducer.class);
		job.setOutputKeyClass(Text.class);
		job.setOutputValueClass(IntWritable.class);
		FileInputFormat.addInputPath(job, new Path(args[0]));
		FileOutputFormat.setOutputPath(job, new Path(args[1]));
		System.exit(job.waitForCompletion(true) ? 0 : 1);

	}
}

In [ ]:
// Desenvolvimento da classe Mapper ContaMaxCidadeProdutoMapper.java

import java.io.IOException;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Mapper;

public class ContaMaxCidadeProdutoMapper extends Mapper<Object, Text, Text, IntWritable> {

	private final static Text cidadeproduto = new Text();

	public void map(Object key, Text value, Context context) throws IOException, InterruptedException {		
		String[] linha=value.toString().split(";");		
		//armazena cidade e produto e uma mesma string (chave) dividida por & e coloca no array (valor) um 1 para cada vez que se repete
		cidadeproduto.set(linha[2] +"&"+linha[3]);
		context.write(cidadeproduto, new IntWritable(1));
	}
}

In [ ]:
// Desesenvolvimento da classe Reducer ContaMaxCidadeProdutoReducer.java

import java.io.IOException;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Reducer;

public class ContaMaxCidadeProdutoReducer extends	Reducer<Text, IntWritable, Text, IntWritable> {
	
	public void reduce(Text key, Iterable<IntWritable> values, Context context) throws IOException, InterruptedException {
		int countValue = 0;
		
		//conta a quantidades de 1 no array de cada chave cidade&produto
		for (IntWritable value : values) {
			countValue += value.get();
		}
		
		context.write(key, new IntWritable(countValue));
	}
}

### 2-Geração do arquivo JAR da aplicação e definição do path:

Exemplo: /home/cloudera/Documents/contamaxcidadeproduto.jar

### 3-Execução do job JAR no ambiente Hadoop e consumo do arquivo bases/compras.txt já armazenado no HDFS:

hadoop jar ~/Documents/contamaxcidadeproduto.jar ContaMaxCidadeProdutoDriver bases/compras.txt saida/cidade_produto_count

### 4-Desenvolvimento das 3 classes Mapper, Reducer e Driver para implementação da 2º iteração MapReduce

In [ ]:
// Desenvolvimento da classe Driver ContaMaxCidadeProdutoSecondDriver.java

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

public class ContaMaxCidadeProdutoSecondDriver {

	public static void main(String[] args) throws Exception {

		Configuration conf = new Configuration();
		Job job = Job.getInstance(conf, "contamaxcidadeprodutoseconddriver");
		job.setJarByClass(ContaMaxCidadeProdutoSecondDriver.class);
		job.setMapperClass(ContaMaxCidadeProdutoSecondMapper.class);
		job.setMapOutputKeyClass(Text.class);
		job.setMapOutputValueClass(Text.class);
		job.setReducerClass(ContaMaxCidadeProdutoSecondReducer.class);
		job.setOutputKeyClass(Text.class);
		job.setOutputValueClass(IntWritable.class);
		FileInputFormat.addInputPath(job, new Path(args[0]));
		FileOutputFormat.setOutputPath(job, new Path(args[1]));
		System.exit(job.waitForCompletion(true) ? 0 : 1);

	}
}

In [ ]:
// Desenvolvimento da classe Mapper ContaMaxCidadeProdutoSecondMapper.java

import java.io.IOException;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Mapper;

public class ContaMaxCidadeProdutoSecondMapper extends Mapper<Object, Text, Text, Text> {

	public void map(Object key, Text value, Context context) throws IOException, InterruptedException{
	    //recebe a linha com duas strings separadas por \t, uma string cidade&produto e outra de soma de 1 para cada chave
		//divide a linha pelo \t 
		String[] linha = value.toString().split("\t");
	    //divide a primeira string da linha pelo & e armazena cidade
		String cidade = linha[0].split("&")[0];
		//divide a primeira string da linha pelo & e armazena produto
	    String produto = linha[0].split("&")[1];
	    //armazena a segunda strin da linha, que é soma dos valores 1
	    String count = linha[1];
	    
	    //envia para o reducer cada chave cidade com o array [produto&valor1, produto&valor2, produto&valor3, ...] 
	    context.write(new Text(cidade), new Text(produto + "&" + count));     
	}
}

In [ ]:
// Desesenvolvimento da classe Reducer ContaMaxCidadeProdutoSecondReducer.java

import java.io.IOException;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Reducer;

public class ContaMaxCidadeProdutoSecondReducer extends	Reducer<Text, Text, Text, Text> {
	
	public void reduce(Text key, Iterable<Text> values, Context context) throws IOException, InterruptedException{
	    int maxVal = Integer.MIN_VALUE;
	    String maxProd = "None";
	    
	    //iterar sobre todos os pruduto&valor de cada cidade/chave
	    for (Text value : values) {
	    	//dividir a string pruduto&valor pelo & e armazena no array ss [produto,valor]
	        String ss []= value.toString().split("&");        
	        //armazena em cnt o valor do produto
	        int cnt = Integer.parseInt(ss[1]);
	        //testa se cnt é maior que o max armazenado, se sim, armazena o novo valor max e o produto
	        if(cnt > maxVal){
	            maxVal = cnt;
	            maxProd = ss[0];
	        }
	    }
	    context.write(key, new Text(maxProd+" " +maxVal));
	}
}

### 5-Geração do arquivo JAR da aplicação e definição do path:

Exemplo: /home/cloudera/Documents/contamaxcidadeprodutosecond.jar

### 6-Execução do job JAR no ambiente Hadoop e consumo do arquivo saida/cidade_produto_count/part-r-00000 resultado do Job anterior e já armazenado no HDFS:

hadoop jar ~/Documents/contamaxcidadeprodutosecond.jar ContaMaxCidadeProdutoSecondDriver saida/cidade_produto_count/part-r-00000  saida/cidade_produto_max